In [1]:
import sys 
import os 

In [2]:
# --------------------------------------------------
def get_tags(season_dict, season, sensor, wf):

    pipeline_out = season_dict[season][sensor][wf]['outputs']['pipeline_out']
    pipeline_tag = season_dict[season][sensor][wf]['outputs']['tag']
    processed_outdir = season_dict[season][sensor][wf]['outputs']['outdir']
   
    return pipeline_out, pipeline_tag, processed_outdir

In [3]:
season_dict = {
    '10': {
        'scanner3DTop': {
            'containers': {
                'gantry_notifications': {
                    'simg': 'gantry_notifications.simg', 
                    'dockerhub_path': 'docker://phytooracle/slack_notifications:latest'
                },
                'preprocessing': {
                    'simg': '3d_preprocessing.simg', 
                    'dockerhub_path': 'docker://phytooracle/3d_preprocessing:latest'
                },
                'sequential_alignment': {
                    'simg': '3d_sequential_align.simg',
                    'dockerhub_path': 'docker://phytooracle/3d_sequential_align:latest'
                },
                'postprocessing': {
                    'simg': '3d_postprocessing.simg',
                    'dockerhub_path': 'docker://phytooracle/3d_postprocessing:latest'
                },
                'plantcrop': {
                    'simg': '3d_crop_individual_plants.simg',
                    'dockerhub_path': 'docker://phytooracle/3d_crop_individual_plants:latest'
                }
            },
            'workflow_1': {
                'commands': [
                'jx2json main_workflow_phase1.jx -a bundle_list.json > main_workflow_phase1.json', 
                'makeflow -T wq --json main_workflow_phase1.json -a -N phytooracle_3d -M phytooracle_3d -r 3 -p 0 -dall -o dall.log $@'
                ],
                'outputs': {
                    'pipeline_out': 'preprocessing_out',
                    'tag': 'preprocessed',
                    'outdir': 'preprocessing'
                }
            },
            'intermediate': {
                'commands': {
                'singularity run 3d_sequential_align.simg -i preprocessing_out/ -o sequential_alignment_out/'
                },
                'outputs': {
                    'pipeline_out': 'sequential_alignment_out',
                    'tag': 'aligned',
                    'outdir': 'alignment'
                }
            },
            'workflow_2': {
                'commands': [
                    'jx2json main_workflow_phase-2.jx -a bundle_list.json > main_workflow_phase2.json', 
                    'makeflow -T wq --json main_workflow_phase2.json -a -r 2 -M phytooracle_3d -N phytooracle_3d -p 60221 -dall -o dall.log --disable-cache $@'
                ],
                'outputs': {
                    'pipeline_out': ['postprocessing_out', 'plant_out'],
                    'tag': ['postprocessed', 'plants'],
                    'outdir': ['postprocessing', 'plantcrop']
                }
            }
        }
    }
}

In [12]:
for item in ['workflow_2']:
    pipeline_out, pipeline_tag, processed_outdir = get_tags(season_dict, '10', 'scanner3DTop', item)

    if type(pipeline_out) == list:
        for i in range(0, len(pipeline_out)):
            print(pipeline_out[i])
            print(pipeline_tag[i])
            print(processed_outdir[i])
            print('\n')

postprocessing_out
postprocessed
postprocessing


plant_out
plants
plantcrop


